In [ ]:
import sys
!{sys.executable} -m pip install scikit-learn

In [ ]:
import sys
!{sys.executable} -m pip install catboost

In [ ]:
import sys
!{sys.executable} -m pip install scikit-optimize

In [ ]:
import sys
!{sys.executable} -m pip install wandb

In [ ]:
import os
os.environ['WANDB_API_KEY'] = 'b3c6dfb61d66930ee7d291c6b19c0ae0b5007c62'

In [ ]:
import wandb
wandb.init(project="exoplanetml", entity="sakhawat",save_code=True,name='CatBoost Classifier',job_type='analysis',reinit=True)

In [ ]:
import numpy as np
np.int = int

In [ ]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [ ]:
from skopt import BayesSearchCV
from catboost import CatBoostClassifier

In [ ]:
# Create an instance of the classifier
catboost_classifier = CatBoostClassifier(grow_policy='Lossguide', loss_function='Logloss', random_seed=43, thread_count=-1)

In [ ]:
# Define the parameter grid for CatBoost
params = {
    'depth': (20, 25),
    'learning_rate': (0.01, 0.5),
    'iterations': (50, 200),
    'num_leaves': (100, 150),
    'min_data_in_leaf': (1, 20),
}


In [ ]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [ ]:
# Create an instance of BayesianSearchCV
bayes_cv = BayesSearchCV(catboost_classifier, params, n_iter=20, cv=cv, n_jobs=-1)

In [ ]:
# Perform hyperparameter tuning using Bayesian optimization
bayes_cv.fit(X_train, np.ravel(y_train))

In [ ]:
# Print the best score and hyperparameters
print("Train score:", bayes_cv.best_score_)
print("Test score:", bayes_cv.score(X_test, y_test))
print("Best hyperparameters:", bayes_cv.best_params_)

In [ ]:
# Log the best score and hyperparameters
wandb.log({"Train score": bayes_cv.best_score_})
wandb.log({"Test score": bayes_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": bayes_cv.best_params_})

In [ ]:
# Finish logging with wandb
wandb.finish()